In [25]:
import arcpy
import datetime
import arcgis
from arcgis.gis import GIS
from arcpy.sa import *

In [2]:
tmp = []
with open(".env") as f:
    for line in f:
        tmp.append(line)
aol_password = tmp[0].split("=")[1].split("\n")[0]
aol_username = tmp[1].split("=")[1].split("\n")[0]

In [3]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [121]:
#arcpy.env.parallelProcessingFactor = "80%"
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb"
arcpy.env.overwriteOutput = True

In [9]:
land_grid = "clipped_land_grid_55km_cea"

In [19]:
for field in arcpy.ListFields(land_grid):
    print(f"{field.name} of type {field.type}")

OBJECTID of type OID
Shape of type Geometry
ID of type Double
AREA_KM2 of type Double
PROP_LAND of type Double
Rich_amph of type Integer
Rar_amph of type Integer
Rich_bird of type Integer
Rar_bird of type Integer
Rich_mamm of type Integer
Rar_mamm of type Integer
Rich_rept of type Integer
Rar_rept of type Integer
Rich_cact of type Integer
Rar_cact of type Integer
Rich_coni of type Integer
Rar_coni of type Integer
Rich_all of type Integer
Rar_all of type Integer
Shape__Area of type Double
Shape__Length of type Double
CELL_ID of type Integer
Shape_Length of type Double
Shape_Area of type Double


In [88]:
raster_cell = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure\\gHM_extended_maskGADM36_cea_20191127.tif"

In [14]:
start = datetime.datetime.now()
arcpy.conversion.FeatureToRaster(land_grid, 
                                 "ID", 
                                 f"{land_grid}_raster", 
                                 raster_cell)
end = datetime.datetime.now() - start
print(f"Finished in {end}")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\WDPA\\WDPA.gdb\\clipped_land_grid_55km_cea_raster'>

In [20]:
start = datetime.datetime.now()
arcpy.sa.ZonalStatisticsAsTable(land_grid, 
                                "CELL_ID", 
                                f"{land_grid}_raster", 
                                f"{land_grid}_raster_ZSaT", 
                                "DATA", "SUM")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:01:44.221876


In [56]:
arcpy.env.extent = "MAXOF"

In [32]:
arcpy.env.snapRaster = f"{land_grid}_raster_ZSaT"

In [33]:
arcpy.env.cellSize = f"{land_grid}_raster_ZSaT"

In [21]:
comm_prot_raster = "wdpa_raisg_comm_20Jan_cea_raster"
not_comm_prot_raster = "wdpa_not_comm_20Jan_cea_raster"

In [41]:
all_prot_raster_name = "prot_all_raster_clipped"

In [36]:
all_prot_raster = Con(IsNull(Raster(not_comm_prot_raster)), 0, 1) + Con(IsNull(Raster(comm_prot_raster)), 0, 1) 

In [37]:
all_prot_raster.save(all_prot_raster_name)

In [40]:
start = datetime.datetime.now()
out_reclass_raster = arcpy.sa.Reclassify(all_prot_raster_name, "Value", "0 NODATA;1 1;2 2", "DATA"); 
out_reclass_raster.save(f"{all_prot_raster_name}_reclass")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:01:32.079852


# because we are working with the land cells, the protected rasters have to be masked using the grid raster

In [57]:
prot_rasters = [f"{all_prot_raster_name}_reclass", comm_prot_raster, not_comm_prot_raster]

In [58]:
for r in prot_rasters:
    start = datetime.datetime.now()
    out_raster = arcpy.sa.ExtractByMask(r, f"{land_grid}_raster")
    out_raster.save(f"{r}_mask")
    end = datetime.datetime.now() - start
    print(f"{r} finished in {end}")

prot_all_raster_clipped_reclass finished in 0:01:32.593411
wdpa_raisg_comm_20Jan_cea_raster finished in 0:01:17.158638
wdpa_not_comm_20Jan_cea_raster finished in 0:02:05.142429


In [61]:
prot_mask_rasters = []
for r in prot_rasters:    
    prot_mask_rasters.append(f"{r}_mask") 

In [62]:
prot_mask_rasters

['prot_all_raster_clipped_reclass_mask',
 'wdpa_raisg_comm_20Jan_cea_raster_mask',
 'wdpa_not_comm_20Jan_cea_raster_mask']

In [59]:
comm_prot_ZSaT = f"comm_prot_raster_clipped_ZSaT"
not_comm_prot_ZSaT = f"not_comm_prot_raster_clipped_ZSaT"
all_prot_ZSaT = f"all_prot_raster_clipped_ZSaT"

In [117]:
zsa_tables = [all_prot_ZSaT, comm_prot_ZSaT, not_comm_prot_ZSaT]

In [118]:
raster_table_dict = dict(zip(prot_mask_rasters, zsa_tables))

In [119]:
raster_table_dict

{'prot_all_raster_clipped_reclass_mask': 'all_prot_raster_clipped_ZSaT',
 'wdpa_raisg_comm_20Jan_cea_raster_mask': 'comm_prot_raster_clipped_ZSaT',
 'wdpa_not_comm_20Jan_cea_raster_mask': 'not_comm_prot_raster_clipped_ZSaT'}

In [120]:
for r in raster_table_dict:
    print(f"{raster_table_dict[r]} is the table and {r} is the masked raster")

all_prot_raster_clipped_ZSaT is the table and prot_all_raster_clipped_reclass_mask is the masked raster
comm_prot_raster_clipped_ZSaT is the table and wdpa_raisg_comm_20Jan_cea_raster_mask is the masked raster
not_comm_prot_raster_clipped_ZSaT is the table and wdpa_not_comm_20Jan_cea_raster_mask is the masked raster


In [122]:
for r in raster_table_dict:
    start = datetime.datetime.now()
    arcpy.sa.ZonalStatisticsAsTable(land_grid, 
                                "CELL_ID", 
                                r, 
                                raster_table_dict[r], 
                                "DATA", "SUM")
    end = datetime.datetime.now() - start
    print(f"Finished in {end}")

Finished in 0:01:49.927671
Finished in 0:01:49.345450
Finished in 0:01:53.275861


## join field for the ZSaT, rename columns, make null as 0 and calculate proportion

In [71]:
land_grid_ZSaT = f"{land_grid}_raster_ZSaT"

In [141]:
land_grid_ZSaT

'clipped_land_grid_55km_cea_raster_ZSaT'

In [123]:
zsa_tables

['all_prot_raster_clipped_ZSaT',
 'comm_prot_raster_clipped_ZSaT',
 'not_comm_prot_raster_clipped_ZSaT']

In [147]:
ZSaTGrid = land_grid_ZSaT
for tbl in zsa_tables:
    pos = len(arcpy.ListFields(ZSaTGrid))
    start = datetime.datetime.now()
    arcpy.management.JoinField(ZSaTGrid, "CELL_ID", tbl, "CELL_ID", "COUNT")
    prot_class=tbl.split("_r")[0]
    newField_name = f"COUNT_{prot_class}" 
    #print(newField_name)
    oldField_name = arcpy.ListFields(ZSaTGrid)[pos].name 
    arcpy.AlterField_management(ZSaTGrid, oldField_name, newField_name, newField_name)
    end = datetime.datetime.now() - start
    print(f"{tbl} finished in {end}")

all_prot_raster_clipped_ZSaT finished in 0:00:08.007377
comm_prot_raster_clipped_ZSaT finished in 0:00:04.012319
not_comm_prot_raster_clipped_ZSaT finished in 0:00:07.983380


In [148]:
for field in arcpy.ListFields(ZSaTGrid):
    print(f"{field.name} of type {field.type}")

OBJECTID of type OID
CELL_ID of type Integer
COUNT of type Integer
AREA of type Double
SUM of type Double
COUNT_all_prot of type Integer
COUNT_comm_prot of type Integer
COUNT_not_comm_prot of type Integer


In [149]:
fieldObs = arcpy.ListFields(ZSaTGrid)  
fieldNames = []
for field in fieldObs:
    fieldNames.append(field.name)
del fieldObs
fieldCount = len(fieldNames)

In [150]:
start = datetime.datetime.now()
with arcpy.da.UpdateCursor(ZSaTGrid, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)
del curU
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:07.973383


In [151]:
for tbl in zsa_tables:
    prot_class=tbl.split("_r")[0]
    prop_name = f"prop_{prot_class}"
    start = datetime.datetime.now()
    arcpy.management.AddField(ZSaTGrid, 
                          prop_name, 
                          "DOUBLE", 
                          None, None, None, 
                          prop_name, "NULLABLE", "NON_REQUIRED", '')
    COUNT_name = f"COUNT_{prot_class}" 
    py_formula = f"(!{COUNT_name}! / !COUNT!) * 100"
    arcpy.management.CalculateField(ZSaTGrid, 
                                prop_name, 
                                py_formula, 
                                "PYTHON3", '') 
    end = datetime.datetime.now() - start
    print(f"{prot_class} finished in {end}")

all_prot finished in 0:00:33.379596
comm_prot finished in 0:00:34.759654
not_comm_prot finished in 0:00:31.911501


In [152]:
fieldNames = []
for tbl in zsa_tables:
    prot_class=tbl.split("_r")[0]
    prop_name = f"prop_{prot_class}"
    fieldNames.append(prop_name)
fieldCount = len(fieldNames)

In [153]:
fieldNames

['prop_all_prot', 'prop_comm_prot', 'prop_not_comm_prot']

In [154]:
for field in arcpy.ListFields(ZSaTGrid):
    print(field.name)

OBJECTID
CELL_ID
COUNT
AREA
SUM
COUNT_all_prot
COUNT_comm_prot
COUNT_not_comm_prot
prop_all_prot
prop_comm_prot
prop_not_comm_prot


In [155]:
start = datetime.datetime.now()
with arcpy.da.UpdateCursor(ZSaTGrid, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] > 100:  
                rowU[field] = "100"  
        curU.updateRow(rowU)
del curU
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:05.518259


In [156]:
field_names_dict = dict(zip(fieldNames, ["all_prot_prop","comm_prot_prop","not_comm_prot_prop"]))

In [157]:
field_names_dict

{'prop_all_prot': 'all_prot_prop',
 'prop_comm_prot': 'comm_prot_prop',
 'prop_not_comm_prot': 'not_comm_prot_prop'}

In [158]:
for old in field_names_dict:
    newField_name = field_names_dict[old]
    #print(newField_name)
    oldField_name =  old
    arcpy.AlterField_management(ZSaTGrid, oldField_name, newField_name, newField_name)
    #print(f"{oldField_name} is the old name, {newField_name} is the new one")

In [159]:
ZSaTGrid

'clipped_land_grid_55km_cea_raster_ZSaT'

In [ ]:
#arcpy.management.AddJoin("clipped_land_grid_55km_cea", 
                         "CELL_ID", 
                         "clipped_land_grid_55km_cea_raster_ZSaT", 
                         "CELL_ID", "KEEP_ALL")

In [ ]:
#arcpy.conversion.FeatureClassToFeatureClass("clipped_land_grid_55km_cea", 
                                            r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb", 
                                            "clipped_land_grid_55km_prot_prop", '', 
                                            'ID "ID" true true false 8 Double 0 0,First,#,clipped_land_grid_55km_cea,clipped_land_grid_55km_cea.ID,-1,-1;all_prot_prop "all_prot_prop" true true false 8 Double 0 0,First,#,clipped_land_grid_55km_cea,clipped_land_grid_55km_cea_raster_ZSaT.all_prot_prop,-1,-1;comm_prot_prop "comm_prot_prop" true true false 8 Double 0 0,First,#,clipped_land_grid_55km_cea,clipped_land_grid_55km_cea_raster_ZSaT.comm_prot_prop,-1,-1;not_comm_prot_prop "not_comm_prot_prop" true true false 8 Double 0 0,First,#,clipped_land_grid_55km_cea,clipped_land_grid_55km_cea_raster_ZSaT.not_comm_prot_prop,-1,-1', 
                                            '')

# marine grid

In [160]:
marine_grid_raster = "marine_grid_raster"

In [161]:
marine_grid = "marine_grid_cea"

In [162]:
arcpy.env.snapRaster = marine_grid_raster

In [163]:
arcpy.env.extent = "MAXOF"

In [164]:
arcpy.env.cellSize = marine_grid_raster

In [165]:
for field in arcpy.ListFields(marine_grid):
    print(f"{field.name} of type {field.type}")

OBJECTID of type OID
Shape of type Geometry
CELL_ID of type String
PROP_LAND of type Double
ISISLAND of type Double
AGRICULTURE of type Double
URBAN of type Double
RAINFED of type Double
AVE_RSR_PC of type Double
SR_PC of type Double
FOCAL_SPP of type String
AREA_KM2 of type Double
Shape_Length of type Double
Shape_Area of type Double


In [112]:
start = datetime.datetime.now()
arcpy.sa.ZonalStatisticsAsTable(marine_grid, 
                                "CELL_ID", 
                                marine_grid_raster, 
                                f"{marine_grid}_raster_ZSaT", 
                                "DATA", "SUM")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:03:04.972732


## masking the protected rasters to marine

In [113]:
prot_rasters = [f"{all_prot_raster_name}_reclass", comm_prot_raster, not_comm_prot_raster]

In [114]:
for r in prot_rasters:
    start = datetime.datetime.now()
    out_raster = arcpy.sa.ExtractByMask(r, marine_grid_raster)
    out_raster.save(f"{r}_marine_mask")
    end = datetime.datetime.now() - start
    print(f"{r} finished in {end}")

prot_all_raster_clipped_reclass finished in 0:01:17.041520
wdpa_raisg_comm_20Jan_cea_raster finished in 0:01:16.838839
wdpa_not_comm_20Jan_cea_raster finished in 0:01:47.865899


In [115]:
prot_marine_mask_rasters = []
for r in prot_rasters:    
    prot_marine_mask_rasters.append(f"{r}_marine_mask") 

In [166]:
prot_marine_mask_rasters

['prot_all_raster_clipped_reclass_marine_mask',
 'wdpa_raisg_comm_20Jan_cea_raster_marine_mask',
 'wdpa_not_comm_20Jan_cea_raster_marine_mask']

In [167]:
comm_prot_marine_ZSaT = f"comm_prot_marine_raster_clipped_ZSaT"
not_comm_prot_marine_ZSaT = f"not_comm_prot_marine_raster_clipped_ZSaT"
all_prot_marine_ZSaT = f"all_prot_marine_raster_clipped_ZSaT"

In [169]:
zsa_marine_tables = [all_prot_marine_ZSaT, comm_prot_marine_ZSaT, not_comm_prot_marine_ZSaT ]

In [170]:
marine_raster_table_dict = dict(zip(prot_marine_mask_rasters, zsa_marine_tables))

In [172]:
marine_raster_table_dict

{'prot_all_raster_clipped_reclass_marine_mask': 'all_prot_marine_raster_clipped_ZSaT',
 'wdpa_raisg_comm_20Jan_cea_raster_marine_mask': 'comm_prot_marine_raster_clipped_ZSaT',
 'wdpa_not_comm_20Jan_cea_raster_marine_mask': 'not_comm_prot_marine_raster_clipped_ZSaT'}

In [173]:
for r in marine_raster_table_dict:
    start = datetime.datetime.now()
    arcpy.sa.ZonalStatisticsAsTable(marine_grid, 
                                "CELL_ID", 
                                r, 
                                marine_raster_table_dict[r], 
                                "DATA", "SUM")
    end = datetime.datetime.now() - start
    print(f"Finished in {end}")

Finished in 0:03:00.912201
Finished in 0:02:57.190449
Finished in 0:03:00.052961


## join field for the ZSaT, rename columns, make null as 0 and calculate proportion

In [174]:
marine_grid_ZSaT = f"{marine_grid}_raster_ZSaT"

In [175]:
zsa_marine_tables

['all_prot_marine_raster_clipped_ZSaT',
 'comm_prot_marine_raster_clipped_ZSaT',
 'not_comm_prot_marine_raster_clipped_ZSaT']

In [177]:
ZSaTGrid = marine_grid_ZSaT
for tbl in zsa_marine_tables:
    pos = len(arcpy.ListFields(ZSaTGrid))
    start = datetime.datetime.now()
    arcpy.management.JoinField(ZSaTGrid, "CELL_ID", tbl, "CELL_ID", "COUNT")
    prot_class=tbl.split("_m")[0]
    newField_name = f"COUNT_{prot_class}" 
    #print(newField_name)
    oldField_name = arcpy.ListFields(ZSaTGrid)[pos].name 
    arcpy.AlterField_management(ZSaTGrid, oldField_name, newField_name, newField_name)
    end = datetime.datetime.now() - start
    print(f"{tbl} finished in {end}")

all_prot_marine_raster_clipped_ZSaT finished in 0:00:09.482450
comm_prot_marine_raster_clipped_ZSaT finished in 0:00:06.217860
not_comm_prot_marine_raster_clipped_ZSaT finished in 0:00:10.240486


In [178]:
for field in arcpy.ListFields(ZSaTGrid):
    print(f"{field.name} of type {field.type}")

OBJECTID of type OID
CELL_ID of type String
ZONE_CODE of type Integer
COUNT of type Integer
AREA of type Double
SUM of type Double
COUNT_all_prot of type Integer
COUNT_comm_prot of type Integer
COUNT_not_comm_prot of type Integer


In [179]:
fieldObs = arcpy.ListFields(ZSaTGrid)  
fieldNames = []
for field in fieldObs:
    fieldNames.append(field.name)
del fieldObs
fieldCount = len(fieldNames)

In [180]:
start = datetime.datetime.now()
with arcpy.da.UpdateCursor(ZSaTGrid, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)
del curU
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:28.544350


In [181]:
for tbl in zsa_marine_tables:
    prot_class=tbl.split("_m")[0]
    prop_name = f"prop_{prot_class}"
    start = datetime.datetime.now()
    arcpy.management.AddField(ZSaTGrid, 
                          prop_name, 
                          "DOUBLE", 
                          None, None, None, 
                          prop_name, "NULLABLE", "NON_REQUIRED", '')
    COUNT_name = f"COUNT_{prot_class}" 
    py_formula = f"(!{COUNT_name}! / !COUNT!) * 100"
    arcpy.management.CalculateField(ZSaTGrid, 
                                prop_name, 
                                py_formula, 
                                "PYTHON3", '') 
    end = datetime.datetime.now() - start
    print(f"{prot_class} finished in {end}")

all_prot finished in 0:01:43.789876
comm_prot finished in 0:01:38.532781
not_comm_prot finished in 0:01:38.287397


In [184]:
fieldNames = []
for tbl in zsa_marine_tables:
    prot_class=tbl.split("_m")[0]
    prop_name = f"prop_{prot_class}"
    fieldNames.append(prop_name)
fieldCount = len(fieldNames)

In [185]:
fieldNames

['prop_all_prot', 'prop_comm_prot', 'prop_not_comm_prot']

In [186]:
for field in arcpy.ListFields(ZSaTGrid):
    print(field.name)

OBJECTID
CELL_ID
ZONE_CODE
COUNT
AREA
SUM
COUNT_all_prot
COUNT_comm_prot
COUNT_not_comm_prot
prop_all_prot
prop_comm_prot
prop_not_comm_prot


In [187]:
start = datetime.datetime.now()
with arcpy.da.UpdateCursor(ZSaTGrid, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] > 100:  
                rowU[field] = "100"  
        curU.updateRow(rowU)
del curU
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:17.080931


In [188]:
field_names_dict = dict(zip(fieldNames, ["all_prot_prop","comm_prot_prop","not_comm_prot_prop"]))

In [189]:
field_names_dict

{'prop_all_prot': 'all_prot_prop',
 'prop_comm_prot': 'comm_prot_prop',
 'prop_not_comm_prot': 'not_comm_prot_prop'}

In [190]:
for old in field_names_dict:
    newField_name = field_names_dict[old]
    #print(newField_name)
    oldField_name =  old
    arcpy.AlterField_management(ZSaTGrid, oldField_name, newField_name, newField_name)
    #print(f"{oldField_name} is the old name, {newField_name} is the new one")

In [191]:
ZSaTGrid

'marine_grid_cea_raster_ZSaT'

In [ ]:
arcpy.management.AddJoin("marine_grid_cea", "CELL_ID", "marine_grid_cea_raster_ZSaT", "CELL_ID", "KEEP_ALL")

In [ ]:
arcpy.conversion.FeatureClassToFeatureClass("marine_grid_cea", 
                                            r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb", 
                                            "clipped_marine_grid_55km_prot_prop", '', 
                                            'CELL_ID "CELL_ID" true true false 254 Text 0 0,First,#,marine_grid_cea,marine_grid_cea.CELL_ID,0,254;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,marine_grid_cea,marine_grid_cea.Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,marine_grid_cea,marine_grid_cea.Shape_Area,-1,-1;all_prot_prop "all_prot_prop" true true false 8 Double 0 0,First,#,marine_grid_cea,marine_grid_cea_raster_ZSaT.all_prot_prop,-1,-1;comm_prot_prop "comm_prot_prop" true true false 8 Double 0 0,First,#,marine_grid_cea,marine_grid_cea_raster_ZSaT.comm_prot_prop,-1,-1;not_comm_prot_prop "not_comm_prot_prop" true true false 8 Double 0 0,First,#,marine_grid_cea,marine_grid_cea_raster_ZSaT.not_comm_prot_prop,-1,-1', '')

# Append land and marine protected proportions
The service has a field named `ID`. The land layer has this field, but the marine one doesn't, furthermore the `CELL_ID` of the marine layer has to be text. 

In [192]:
marine_prot_prop = "clipped_marine_grid_55km_prot_prop"
land_prot_prop = "clipped_land_grid_55km_prot_prop"

In [ ]:
arcpy.conversion.FeatureClassToFeatureClass("clipped_marine_grid_55km_prot_prop", 
                                            r"C:\Users\Vizuser\Documents\ArcGIS\Projects\WDPA\WDPA.gdb", 
                                            "clipped_marine_grid_55km_prot_prop_subset", 
                                            "all_prot_prop > 0 Or comm_prot_prop > 0 Or not_comm_prot_prop > 0", 
                                            'CELL_ID "CELL_ID" true true false 254 Text 0 0,First,#,clipped_marine_grid_55km_prot_prop,CELL_ID,0,254;all_prot_prop "all_prot_prop" true true false 8 Double 0 0,First,#,clipped_marine_grid_55km_prot_prop,all_prot_prop,-1,-1;comm_prot_prop "comm_prot_prop" true true false 8 Double 0 0,First,#,clipped_marine_grid_55km_prot_prop,comm_prot_prop,-1,-1;not_comm_prot_prop "not_comm_prot_prop" true true false 8 Double 0 0,First,#,clipped_marine_grid_55km_prot_prop,not_comm_prot_prop,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,clipped_marine_grid_55km_prot_prop,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,clipped_marine_grid_55km_prot_prop,Shape_Area,-1,-1', '')